
# Thermodynamic Sea-Ice Model

If `params[ice_ON]=True`, the PWP model is able to generate sea-ice. The algorithms for generating sea-ice are located in *PWP_ice.py* file. When sea-ice is generated, the iteration step of the PWP model becomes:

3. Iterate the PWP model:
    + _if sea-ice exists_, compute ocean-ice heat flux, evolve sea-ice and apply sea-ice salinity flux (`PWP_ice.py`). 
    + _if sea-ice does not exist_, apply heat flux to surface ocean and create sea-ice if necessary.
    + apply E-P fluxes.
    + apply wind stress (momentum flux).
    + apply drag associated with internal wave dissipation.
    + apply bulk Richardson mixing.
    + apply gradient Richardson mixing. 
    + apply diapycnal diffusion. 

Currently, the ice-model is purely thermodynamic and does not affect the way wind stress is applied to the ocean. In other words, the model tacitly assumes that the wind momentum is transferred through the sea-ice without any loss - a somewhat reasonable assumption when considering free-drift conditions.


### Creating initial sea ice

The `create_initial_ice()` is called when the temperature of top layer of the PWP model first dips below the freezing point. This function resets the temperature of the top layer to the freezing point of seawater then converts the "excess cooling" into ice. The appropriate ice thickness is determined using this formula

$$
dh_i = \frac{\rho_{sw}\, c_{sw}}{\rho_i\,L_i}\,(T_{fz} - T)\,dz
$$

where $dh_i$ is the thickness of the newly formed ice, $\rho_{sw}$ is a constant reference seawater density, $c_{sw}$ is the specific heat capacity of seawater, $\rho_i$ is a constant reference density of sea-ice, $L_i$ is the latent heat of fusion, $T_{fz}$ is the freezing point of seawater, $T$ is the temperature of the top model layer and $dz$ is the thickness of the top layer.

After the appropriate ice thickness $dh_i$ is computed, the corresponding surface salinity flux is determined using

$$
dS = \alpha\,dh_i\,(S_{sw} - S_i)/dz
$$

where is $dS$ is the salinity change of the top ocean layer, $\alpha$ is the prescribed sea-ice area fraction, $S_{sw}$ is a reference ocean salinity and $S_i$ is sea-ice salinity which is assumed to be 4 psu.

In the `create_initial_ice()` function and the other sea-ice functions, constant reference values are used for ocean salinity, ocean density and ocean freezing point. This is done for simplicity and to ensure conservation of mass. In the real world, the freezing point is a function of salinity.

### Modifying existing sea-ice

After sea-ice is created, one of three functions is used to evolve its thickness: `ice_model_0()`,  `ice_model_T()` and `ice_model_F()`. These models evolve sea-ice then apply the appropriate salinity flux to ocean model's top layer. The default model is `ice_model_T()`.

 

#### Simple sea-ice model
The `ice_model_0()` sea-ice model is analogous to the ice-models used in many studies (e.g. Martinson 1992, Hyatt 2006, Goosse and Zunz 2014). It is a super fast and simple way to produce sea-ice with realistic thickness. To enable this model set `params['iceMod']=0` in `set_params`.

`ice_model_0()` grows ice using the same process that generates initial ice. That is, if the top layer is cooled below the freezing point, the excess cooling is used to generate ice. The amount of ice that is created accumulates with every time step.

If the net heat fluxes to the ice are positive (warming), sea-ice is treated as a slab of ice at uniform temperature. If the ice temperature is below the freezing of seawater, the `ice_model_0()` function first warms the ice to its freezing point using

$$
dT_{ice} = \frac{(F_{oi} + F_{ai})\,dt}{c_i\,h_i\,\rho_i}
$$

where $dT_{ice}$ is the mean ice temperature, $F_{oi}$ is the ocean-ice heat flux, $F_{ai}$ is the atmosphere ice heat flux, $dt$ is the time step, $c_i$ is the specific heat capacity of sea-ice, $h_i$ is ice thickness and $\rho_i$ is ice density (assumed to be constant as before).

If the available heat fluxes are sufficient to start melting ice, the amount of melt is computed as

$$
dh_i = \frac{F_{melt}}{\rho_i\,L_i}\,dt
$$

where $F_{melt}$ is the total available heat flux for ice melt, which is the sum of atmospheric and oceanic heat flux minus the heat used to warm the ice to its freezing point. 

Once $dh_i$ is determined, the appropriate salinity flux is applied to the ocean top layer using equation as earlier.

In the event $F_{oi} + F_{ai}$ exceeds the heat required to completely melt the existing the ice, the excess heat is passed on to the ocean.


#### Sea-ice model with prescribed ice surface temperatures (v1)

![ice-model-schematic](README_plots/PWP_ice_schematic.png)

The `ice_model_T()` model is a slightly more realistic sea-ice model that simulates the heat flux through sea-ice istelf. This is default model used by the ice code and it is loosely based on the toy sea-ice models used in [Thorndike 1992](http://onlinelibrary.wiley.com/doi/10.1029/92JC00695/abstract) and [Eisenman and Wettlaufer 2009](http://www.pnas.org/content/106/1/28.short). In this framework, sea-ice is modeled as a slab of ice with a linear temperature profile as illustrated above. The relevant equations are

$$
\frac{dT_i}{dt} = 2\,\frac{F_{ai}\,+\,F_{oi}}{c_i\,\rho_i\,h_i} 
$$
$$
\frac{dh_i}{dt} = \frac{F_i\,-\,F_{oi}}{L_i\,\rho_i}
$$

where $F_i = -k_i\,\frac{T_{i} - T_{fz}}{h_i}$ is the heat flux through the ice and $k_i$ is the thermal conductivity of sea-ice. Here, $dT_i$ and $T_i$ refer to the *surface ice temperature*. The base of the sea-ice is assumed to be at the freezing point of sea water. If the heat flux from the ice to the atmosphere $F_i$ exceeds the heat flux from the ocean to the ice, sea-ice grows.

Unfortunately, these equations are difficult to solve numerically because they are coupled and non-linear. The main issue is the factor of $h_i$ in the denominator of the temperature equation. To capture the growth rate at very small ice thicknesses requires an extremely small time step ($<<1$ second). 

One way to drastically simplify this model is to prescribe the surface temperature of sea-ice. Doing this eliminates reduces the ice-model to just the thickness equation. The model is still non-linear (because of the inverse relationship between $F_i$ and $h_i$) but it is now much more numerically stable. Because of this non-linearity, the ice model is sub-sampled at a higher time resolutions (default is 5 seconds).

If at some point during the integration the prescribed ice surface temperatures exceed the freezing point, the `ice_model_T()` model assumes sea-ice is warming/melting and reverts to `ice_model_0()`. In other words, `ice_model_T()` is only utilized when the ice-surface temperatures are cold enough to potentially grow the ice.


When running this ice-model, a 1-D array of ice-surface temperatures (K) must be added to the forcing file under the name **skt**. (I typically use skin temperatures from reanalysis for this purpose).

To enable this model set `params['iceMod']=1` in `set_params`. See example.


#### Sea-ice model with prescribed ice surface temperatures (v2)

Another way to simplify the coupled sea-ice model is to prescribe the atmosphere-ice heat flux $F_{ai}$ and assume $F_{ai}=-F_i$. This is equivalent to assuming that the surface of the ice is in thermal equilibrium with the atmosphere. With this relationship, we can solve for the ice surface temperature algebriacally

$$
T_i = -\frac{F_{ai}\,h_i}{k_i}\, -\, T_{fz}
$$

and the ice-model becomes a single 1-D linear equation for thickness.

$$
\frac{dh_i}{dt} = \frac{F_i\,-\,F_{oi}}{L_i\,\rho_i} = \frac{-F_{ai}\,-\,F_{oi}}{L_i\,\rho_i}
$$

In this case, ice-temperature is not (directly) used to compute ice-thickness. This computation is done in the `ice_model_F()` ice-model. To enable this model set `params['iceMod']=2` in `set_params`. See example.

#### Ocean-ice heat flux

The ocean-ice heat flux is computed by the `get_ocean_ice_heat_flux()` in *PWP_ice.py*. The relevant formula is

$$
F_{oi} = c_{sw}\,\rho_{sw}\,c_h\,u^*\,(T_0 - T_{fz})
$$

where $F_{oi}$ is the ocean-ice heat flux, $c_h$ is the heat transfer co-efficient, $u^*$ is the skin friction speed at the ice-ocean interface and $T_0$ is the ocean surface temperature. $c_d=0.0056$ following [McPhee 1992](http://onlinelibrary.wiley.com/doi/10.1029/92JC00239/full) and [McPhee et al. 1999](). Currently, $u^*$ is set to $0.01$ m/s as is done in the [CESM CAM3 slab ocean model](http://www.cesm.ucar.edu/models/atm-cam/docs/description/node37.html). In reality, $u^*$ is dependent on the relative velocity between the ocean surface and the ice; higher relative speeds would cause greater heat-transfer. However, these speeds are generally not available and computing them would require adding a dynamical component to this ice-model. The latter is feasible but not trivial. This may be implemented in the future. 

## Examples

+ [Winter integration over Maud Rise](https://github.com/earlew/pwp_python/blob/with_sea_ice_v2/Example_run_with_sea_ice1.ipynb)